In [82]:
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline


import pandas as pd
import numpy as np
# from fastai.imports import *
# from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [115]:
data = pd.read_csv('Train.csv')
# pd.options.display.max_columns = None  #added to show all the columns inplace of the dots.
data.head(10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,Low,12/18/2008 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,High,8/26/2004 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,High,11/17/2005 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,Low,8/27/2009 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
9,1139275,65000,1016474,3883,121,3.0,1000,20700.0,Medium,8/9/2007 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [114]:
val  = pd.get_dummies(data.Thumb) #One hot encoding to remove categorial data when you have less variation in the feature.
val
# data.SalePrice = np.log(data.SalePrice)

,Hydraulic,Manual,None or Unspecified
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,1
7,0,0,0
8,0,0,1
9,0,0,0


In [4]:
data.SalePrice

0         11.097410
1         10.950807
2          9.210340
3         10.558414
4          9.305651
5         10.184900
6          9.952278
7         10.203592
8          9.975808
9         11.082143
10        10.085809
11        10.021271
12        10.491274
13        10.325482
14        10.239960
15         9.852194
16         9.510445
17         9.159047
18         9.433484
19         9.350102
20        10.621327
21        10.448715
22        10.165852
23        11.198215
24        10.404263
25         9.433484
26         9.648595
27        10.878047
28        10.736397
29        11.396392
            ...    
401095     9.259131
401096     9.210340
401097     9.047821
401098     9.259131
401099     9.305651
401100     9.259131
401101     9.210340
401102     9.259131
401103     9.433484
401104     9.259131
401105     9.210340
401106     9.259131
401107     9.433484
401108     9.259131
401109     9.259131
401110     9.259131
401111     9.210340
401112     9.259131
401113     9.259131


In [4]:
# ?RandomForestRegressor  #to see the documentation
# ??RandomForestRegressor  #to see the code behind the object.
data.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [116]:
#to categorize 'UsageBand' Column.
data[['UsageBand_High','UsageBand_Low','UsageBand_Medium']] = pd.get_dummies(data.UsageBand)
# data[['UsageBand_High','UsageBand_Low','UsageBand_Medium']] = pd.get_dummies(data.UsageBand, columns=['UsageBand_High','UsageBand_Low','UsageBand_Medium'])
data.pop('UsageBand')
data

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saledate,fiModelDesc,...,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,UsageBand_High,UsageBand_Low,UsageBand_Medium
0,1139246,66000,999089,3157,121,3.0,2004,68.0,11/16/2006 0:00,521D,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,0,1,0
1,1139248,57000,117657,77,121,3.0,1996,4640.0,3/26/2004 0:00,950FII,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,0,1,0
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,2/26/2004 0:00,226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,5/19/2011 0:00,PC120-6E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,7/23/2009 0:00,S175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,12/18/2008 0:00,310G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,8/26/2004 0:00,790ELC,...,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,1,0,0
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,11/17/2005 0:00,416D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,8/27/2009 0:00,430HAG,...,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,0,1,0
9,1139275,65000,1016474,3883,121,3.0,1000,20700.0,8/9/2007 0:00,988B,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,0,0,1


In [117]:
# to categorize 'Thumb' column.
data[['Thumb_Hydraulic','Thumb_Manual','Thumb_None']] = pd.get_dummies(data.Thumb)
data.pop('Thumb')
data

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saledate,fiModelDesc,...,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,UsageBand_High,UsageBand_Low,UsageBand_Medium,Thumb_Hydraulic,Thumb_Manual,Thumb_None
0,1139246,66000,999089,3157,121,3.0,2004,68.0,11/16/2006 0:00,521D,...,NaN,NaN,Standard,Conventional,0,1,0,0,0,0
1,1139248,57000,117657,77,121,3.0,1996,4640.0,3/26/2004 0:00,950FII,...,NaN,NaN,Standard,Conventional,0,1,0,0,0,0
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,2/26/2004 0:00,226,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,5/19/2011 0:00,PC120-6E,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,7/23/2009 0:00,S175,...,NaN,NaN,NaN,NaN,0,0,1,0,0,0
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,12/18/2008 0:00,310G,...,NaN,NaN,NaN,NaN,0,1,0,0,0,0
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,8/26/2004 0:00,790ELC,...,NaN,NaN,NaN,NaN,1,0,0,0,0,1
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,11/17/2005 0:00,416D,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,8/27/2009 0:00,430HAG,...,NaN,NaN,NaN,NaN,0,1,0,0,0,1
9,1139275,65000,1016474,3883,121,3.0,1000,20700.0,8/9/2007 0:00,988B,...,NaN,NaN,Standard,Conventional,0,0,1,0,0,0


In [118]:
# pd.get_dummies(data.UsageBand) #One hot encoding to remove categorial data when you have less variation in the feature.

In [49]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)



['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
['cold']


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,cold,hot,warm
0,1,0,0
1,1,0,0
2,0,0,1
3,1,0,0
4,0,1,0
5,0,1,0
6,0,0,1
7,1,0,0
8,0,0,1
9,0,1,0


In [5]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(data.drop('SalePrice', axis=1), data.SalePrice)

ValueError: could not convert string to float: 'Conventional'